# Information Retrieval 1#
## Assignment 2: Retrieval models [100 points] ##

In [197]:
import logging
import sys
import os

def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))

Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.5 example
Q1 Q0 DOC2 3 0.5 example
Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example


In [49]:
import pyndri

index = pyndri.Index('index/')
token2id, id2token, _ = index.get_dictionary()

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [9]:
import collections
import io
import logging
import sys

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

with open('./ap_88_89/topics_title', 'r') as f_topics:
    topics = parse_topics([f_topics])

### Task 1: Implement and compare lexical IR methods [35 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) and 
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[5 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand who the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [10]:
with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

index = pyndri.Index('index/')

num_documents = index.maximum_document() - index.document_base()

dictionary = pyndri.extract_dictionary(index)

tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

# inverted index creation.
document_lengths = {}
unique_terms_per_document = {}

inverted_index = collections.defaultdict(dict)
collection_frequencies = collections.defaultdict(int)
docs_unigram = collections.defaultdict(dict)

total_terms = 0

for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)

    document_bow = collections.Counter(
        token_id for token_id in doc_token_ids
        if token_id > 0)
    document_length = sum(document_bow.values())

    document_lengths[int_doc_id] = document_length
    total_terms += document_length

    unique_terms_per_document[int_doc_id] = len(document_bow)

    for query_term_id in query_term_ids:
        assert query_term_id is not None

        document_term_frequency = document_bow.get(query_term_id, 0)

        if document_term_frequency == 0:
            continue

        #term frequency
        collection_frequencies[query_term_id] += document_term_frequency
        # document frequency
        inverted_index[query_term_id][int_doc_id] = document_term_frequency
        # unigram model (added)
        docs_unigram[int_doc_id][query_term_id] = document_term_frequency

avg_doc_length = total_terms / num_documents    

Gathering statistics about 456 terms.


In [11]:
from math import log

def tfidfScore(doc_id, query):
    """
    TF-IDF scoring function for a document and a query term
    
    :param doc_id: the document id
    :param query: the query (list of query term id's)
    """
    tfidf_score = 0.0
    for t in query:
        try:
            # tf = how often does term appear in document
            tf = inverted_index[t][doc_id]
            idf = log(num_documents) - log(len(inverted_index[t]))
            tfidf_score += log(1+tf)*idf
        except KeyError:
            pass
        
    return tfidf_score

In [12]:
def BM25Score(doc_id, query, k1=1.2, b=0.75):
    """
    BM25 scoring function for a document and a query
    
    :param int_document_id: the document id
    :param query: the query (list of terms)
    :param k1: (fixed at 1.2)
    :param b: (fixed at 0.75)
    """
    bm25 = 0.0
    for q in set(query):
        try:
            tf = inverted_index[q][doc_id]
            w = ((k1+1)*tf) / (k1*((1-b)+b*(avg_doc_length))+tf)
            idf = log(num_documents) - log(len(inverted_index[q]))
            # use logs to prevent underflow
            bm25 += w + idf 

        except KeyError:
            pass
        
    return bm25

In [16]:
def constructLM(document_term_freqs, collection_term_freqs, smoothing, param):
    '''
    Constructs language model using several different smoothing techniques
    
    Jelinek-Mercer smoothing linearly interpolates using the background language model.
    Intuively, this captures the assumption that for an unseen word, its smoothed
    probability should depend on the frequency of its appearence over all documents. 
    LM is a multinomial distrubution (unigram). Param=0 uses only the background LM
    
    Dirichlet smoothing computes doc-term probabilities using both the document LM
    as well as the background LM, and does so while taking into account
    the document's length. The intuition behind this is that longer
    documents need less smoothing because they can rely more on their own LM.
    
    Absolute discounting substracts param from each document term frequency count
    and adds it to unseen words, thus redistributes the probability mass. 
    
    :param document_term_freq: the document term frequencies 
    :param collection_term_freq: the collection term frequencies used for background LM
    :param smoothing: different types of smoothing
    :param param: decides the degree of smoothing
    '''
    n_doc_tokens = sum(document_term_freqs.values())
    n_doc_types = sum(set(document_term_freqs.values()))
    n_col_tokens = sum(collection_term_freqs.values())

    document_term_freqs = collections.defaultdict(int,document_term_freqs)
    doc_term_prob = collections.defaultdict(float)
    
    # Loop over vocabulary (query terms) and construct JM-smoothed document language model
    for q_t_id in query_term_ids:
        doc_tf = document_term_freqs[q_t_id]
        col_tf = collection_term_freqs[q_t_id]
        
        if smoothing == 'JM':
            try:
                doc_term_prob[q_t_id] = param*(doc_tf/float(n_doc_tokens)) + (1-param)*(col_tf/n_col_tokens)
            except ZeroDivisionError:
                return None
                
        elif smoothing == 'dirichlet':
            doc_term_prob[q_t_id] = (doc_tf + param*(col_tf/n_col_tokens)) / (n_doc_tokens+param)
            
        elif smoothing == 'discount':
            try:
                doc_term_prob[q_t_id] = max(doc_tf-param,0)/n_doc_tokens + (param*n_doc_types)/n_doc_tokens * (col_tf/n_col_tokens)
            except ZeroDivisionError:
                return None
        else:
            raise('Smoothing method not recognized!')

    return doc_term_prob

def queryLikelihood(query_id, LM):
    '''
    Calculates query likelihood given language model and query
    
    :param query_id: query id
    :param LM language model
    '''
    if LM is None:
        return None
    
    q_likelihood = 0
    for q_t_id in tokenized_queries[query_id]:
        try:
            q_likelihood += log(LM[q_t_id])
        except ValueError:
            pass

    return q_likelihood

-45.500192454108415


In [161]:
import math
from scipy.stats import norm

COLLECTION_FREQUENCY = sum(collection_frequencies.values())

def gaussian(i, j, sigm=50):
    """Implements proximity-based Gaussian kernel function"""
    return math.exp(-(i-j)**2/(2*sigm**2))

def triangle(i, j, sigm=50):
    """Implements proximity-based Triangle kernel function"""
    if abs(i-j) <= sigm:
        value = 1 - abs(i-j)/sigm
    else:
        value = 0
    return value

def cosine(i, j, sigm=50):
    """Implements proximity-based Cosine (Hamming) kernel function"""
    if abs(i-j) <= sigm:
        value = 0.5*(1 + math.cos(abs(i-j)*math.pi/sigm))
    else:
        value = 0
    return value    

def circle(i, j, sigm=50):
    """Implements proximity-based Circle kernel function"""
    if abs(i-j) <= sigm:
        value = (1 + cos(abs(i-j)*math.pi/sigm))
    else:
        value = 0
    return value     
    
def passage(i, j, sigm=50):
    """Implements proximity-based Passage kernel function"""
    if abs(i-j) <= sigm:
        value = 1
    else:
        value = 0
    return value

def PLM_prob(int_document_id, query_term_id, i, col_lang_model, ocs, mu=50, kernel=gaussian):
    """Calculates estimated language model of virtual document"""
    _, doc_token_ids = index.document(int_document_id)
    prop_count = sum([kernel(i,j) for j in ocs])
    N = document_lengths[int_document_id]
    
    sigma = 50
    if kernel == gaussian:
        z_i = math.sqrt(2*math.pi*sigma**2)*(norm.cdf((N-i)/float(sigma) -
                                                      norm.cdf((1-i)/sigma)))
    elif kernel == triangle:
        z_i = sigma
    elif kernel == cosine:
        z_i = sigma
    elif kernel == circle:
        raise NotImplementedError
    elif kernel == passage:
        z_i = 2*sigma
    
    prob = (prop_count + mu * col_lang_model[query_term_id]/(float(z_i) + mu))
    return prob

def PLM_score(int_document_id, query, i, col_lang_model, query_lang_model, ocs, mu=50, kernel=gaussian):
    """Calculates score for PLM at position i"""
    score = 0
    for query_term_id in set(query):
        #ML Estimate of query language model        
        score -= query_lang_model[query_term_id] * math.log(query_lang_model[query_term_id]/PLM_prob(int_document_id, 
                                                                     query_term_id, 
                                                                     i, col_lang_model, ocs, mu=mu,
                                                                     kernel=kernel))
    return score
            
def PLM(int_document_id, query, mu=50,kernel=triangle):
    """Scores document using PLM Best Position Strategy"""
    N = document_lengths[int_document_id]
    _, doc_token_ids = index.document(int_document_id)
    best_score = 0
    
    query_lang_model = {}
    col_lang_model = {}
    ocs = {}
    query_len = len(query)
    for q in query:
        query_lang_model[q] = query.count(q)/float(query_len)
        col_lang_model[q] = collection_frequencies[q
                                                   ]/COLLECTION_FREQUENCY
        ocs[q] = [i+1 for i, x in enumerate(doc_token_ids) if x == q]
    
    for i in range(1,N+1):
        score = PLM_score(int_document_id, query, i, col_lang_model, query_lang_model, ocs, mu=mu, kernel=kernel)
        if score > best_score:
            best_score = score
            
    return best_score

In [162]:
def run_retrieval(model_name, score_fn, param=0.1):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example)
    :param param: parameter value for score_fn
    """
    run_out_path = '{}.run'.format(model_name+str(param))

    if model_name == 'lsi' or model_name == 'lsa':
        print('creating LSM...')
        lsm = LSM(model_name)
        print('created LSM.')
    
    print('Retrieving using', model_name,'...')
    data = collections.defaultdict(list)  
    
    i=5000
    for doc_id in range(index.document_base(), index.maximum_document()):
        if doc_id % i == 0: print("Percentage completed: %.2f" % (doc_id/index.maximum_document()))
        
        if model_name == 'JM' or model_name == 'dirichlet' or model_name == 'discount':
            LM = constructLM(docs_unigram[doc_id], collection_frequencies, model_name, param)
         
        for query_id, query_terms in tokenized_queries.items():
            if model_name == 'tfidf' or model_name == 'BM25':
                score = score_fn(doc_id, tokenized_queries[query_id])

            # smoothing
            elif model_name == 'JM' or model_name == 'dirichlet' or model_name == 'discount':
                score = score_fn(query_id, LM)

            #use external document id (index.document(doc_ID))
            if score is not None:
                data[query_id].append((score,index.document(doc_id)[0]))
    
    print('Retrieving completed.')
    
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)

In [189]:
from collections import defaultdict

# translate document names to document index
doc_id_transl = {}
for i in range(1,index.document_count()):
    doc_id_transl[index.ext_document_id(i)] = i

# load top 1000 tf-idf
with open('run/tfidf.run') as f:
    top_1000 = defaultdict(list)
    for line in f:
        line = line.split()
        q_id, doc_name = line[0], line[2]
        top_1000[q_id].append(doc_id_transl[doc_name])

def run_1000(method, score_fn, fn, param, max_length=1000):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    Similar to run_retrieval, but uses top 1000 (or other specified value) tfidf documents.
    Used for LSI, LDA and PLM.
    
    :param method: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example)
    :param param: parameter value for score_fn
    :param max_length: 
    """
    if method == 'lsi' or method == 'lda':
        lsm = LSM(method, num_topics=param)
    
    i = 1
    data = collections.defaultdict(list)
    for query_id in top_1000.keys():
        print('query {} from {}'.format(i, len(top_1000)))
        i+=1
        j = 0
        for doc_id in top_1000[query_id][:max_length]:
            if j % 25 == 0:
                print('docs: {}%'.format((j/100)*100))
            j+=1
            if method == 'lsi' or method == 'lda':
                score = LSM_score(query_id, doc_id, lsm, method)
            
            # check if query term in document, otherwise skip
            elif method == 'PLM':
                calculate_score = False
                for q in tokenized_queries[query_id]:
                    if q in set(index.document(doc_id)[1]):
                        calculate_score = True
                        break
                
                if calculate_score == True:
                    score = score_fn(doc_id, tokenized_queries[query_id], mu=param)
                else:
                    score = None
                
            if score is not None:
                data[query_id].append((score,index.document(doc_id)[0]))
                
    print('Retrieving completed.')

    with open(fn, 'w') as f_out:
        write_run(
            model_name=method,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)

In [187]:
import numpy as np

def trec_eval(qrel, run, mean=True):
    """
    Python wrapper function for trec_eval that uses shell scripting to obtain
    NDCG@10, MAP@1000, Precision@5 and Recall@1000 scores. Returns dict with scores.
        
    :param qrel: the location of the qrel file to use
    :param run: the location of the .run file to use
    :param mean: if True, only the mean scores are returned.
                 Otherwise, lists with scores per query are returned.
    """
    scores = {}
    
    nDCG = !trec_eval/./trec_eval -m all_trec -q {qrel} {run} | grep -E "^ndcg_cut_10 "
    scores['nDCG'] = np.array([float(s.split()[-1]) for s in nDCG[:-1]])
    MAP = !trec_eval/./trec_eval -m all_trec -q {qrel} {run} | grep -E "^map\s"            
    scores['MAP'] = np.array([float(s.split()[-1]) for s in MAP[:-1]])
    P5 = !trec_eval/./trec_eval -m all_trec -q {qrel} {run} | grep -E "^P_5\s"
    scores['P5'] = np.array([float(s.split()[-1]) for s in P5[:-1]])
    recall = !trec_eval/./trec_eval -m all_trec -q {qrel} {run} | grep -E "^recall_1000\s"
    scores['recall'] = np.array([float(s.split()[-1]) for s in recall[:-1]])
    
    if mean == True:
        scores['nDCG'] = np.mean(scores['nDCG'])
        scores['MAP'] = np.mean(scores['MAP'])
        scores['P5'] = np.mean(scores['P5'])
        scores['recall'] = np.mean(scores['recall'])

    return scores

# How to use

In [ ]:
run_retrieval('lsi', LSM_score)

In [63]:
trec_eval('ap_88_89/qrel_validation', 'tfidf.run')

{'MAP': '0.2195',
 'P5': '0.3467',
 'nDCG': '0.3586',
 'recall': 0.6239876590821443}

In [196]:
# JM range 0.1-0.9
for p in np.arange(0.1,1.0,0.1):
    run_retrieval('JM', queryLikelihood, param=p)

Retrieving using JM ...
Percentage completed: 0.03
Percentage completed: 0.06
Percentage completed: 0.09
Percentage completed: 0.12
Percentage completed: 0.15
Percentage completed: 0.18
Percentage completed: 0.21
Percentage completed: 0.24
Percentage completed: 0.27
Percentage completed: 0.30
Percentage completed: 0.33
Percentage completed: 0.36
Percentage completed: 0.39
Percentage completed: 0.43
Percentage completed: 0.46
Percentage completed: 0.49
Percentage completed: 0.52
Percentage completed: 0.55
Percentage completed: 0.58
Percentage completed: 0.61
Percentage completed: 0.64
Percentage completed: 0.67
Percentage completed: 0.70
Percentage completed: 0.73
Percentage completed: 0.76
Percentage completed: 0.79
Percentage completed: 0.82
Percentage completed: 0.85
Percentage completed: 0.88
Percentage completed: 0.91
Percentage completed: 0.94
Percentage completed: 0.97
Retrieving completed.
Retrieving using JM ...
Percentage completed: 0.03
Percentage completed: 0.06
Percentage c

## Plots

For the language models, create plots showing NDCG@10 with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's subprocess.

## Two-tailed paired Student t-test
Compute significance of the results using a two-tailed paired Student t-test [5 points]. Be wary of false rejection of the null hypothesis caused by the multiple comparisons problem. There are multiple ways to mitigate this problem and it is up to you to choose one.

In [ ]:
from scipy.stats import ttest_rel

### Task 2: Latent Semantic Models (LSMs) [15 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

In [167]:
class MyCorpus:
    '''
    This class creates a generator object from a pyndri index, yielding
    a document represented as a bow or dense tfidf compatible with gensim.
   
    :param: method: which method to use, if lda than return a bow and otherwise
            return a dense tfidf. If word2vec than return list of words per doc.
    '''
    def __init__(self, method):
        self.method = method

    def __iter__(self):
        for int_doc_id in range(index.document_base(), index.maximum_document()):
            unigram = docs_unigram[int_doc_id]
           
            # return BOW representation
            if self.method == 'lda':
                yield list(unigram.items())
           
            # return TFIDF representation
            elif self.method == 'lsi':
                yield dense_tfidf(unigram)
            elif self.method == 'word2vec':
                ext_doc_id, doc_token_ids = index.document(int_doc_id)
                yield tuple(dictionary[token_id] for token_id in doc_token_ids \
                            if token_id > 0 and token_id in dictionary)
            else:
                raise ValueError('Method in MyCorpus undefined!')

In [18]:
def dense_tfidf(unigram):
    '''
    Transforms a document with unigram representation to a dense 
    TFIDF list. Returns a list of (token_id, tfidf) tuples.
    '''
    tfidf_dict = collections.defaultdict(float)
    for token_id, tf in unigram.items():
        idf = log(num_documents) - log(len(inverted_index[token_id]))
        tfidf_dict[token_id] = log(1+tf)*idf
    return list(tfidf_dict.items())

In [89]:
from gensim.models import LsiModel
from gensim.models import LdaModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def LSM(lsm_method, num_topics=10):
    '''
    Implements two Latent Sematic Models: LSI and LDA using gensim.
    
    :param method: either lsi or lda. This decides whether the
    corpus streaming object returns a BOW or TFIDF representation
    and whether the latent semantic model that is being returned
    is LSI or LDA.
    '''
    print('creating LSM with '+str(num_topics)+' topics.')
    
    corpus = MyCorpus(lsm_method)
    if lsm_method == 'lsi':
        lsm = LsiModel(corpus, id2word = id2token, num_topics=num_topics)
    else:
        lsm = LdaModel(list(corpus), id2word = id2token, num_topics=num_topics)
    return lsm

def LSM_score(query_id, document_id, lsm, lsm_method):
    '''
    Calculate a query-document score between either LSA 
    or LDA representations of the query and document
    using the cosine similarity.
    '''
    
    # Construct query and and document representation
    query_unigram = collections.Counter(
        token_id for token_id in tokenized_queries[query_id]
        if token_id > 0)
    doc_unigram = docs_unigram[document_id]
    
    if lsm_method == 'lsi':
        query_repr = lsm[dense_tfidf(query_unigram)]
        doc_repr = lsm[dense_tfidf(doc_unigram)]
    else:
        query_repr = lsm[list(query_unigram.items())]
        doc_repr = lsm[list(doc_unigram.items())]

    query_repr = [[topic for idx, topic in query_repr]]
    doc_repr = [[topic for idx, topic in doc_repr]]
    
    try:
        score = cosine_similarity(query_repr, doc_repr)
    except ValueError:
        return None
        
    return score[0][0]

In [245]:
for param in [10, 20, 50, 100, 200,400,600,800,1000]:
    print('creating file: run/lda_ntopics='+str(param)+'.run')
    run_1000(top_1000, doc_repres, 'lda', param, 'lda_ntopics='+str(param)+'.run')
        print('creating file: run/lsi_ntopics='+str(param)+'.run')
    run_1000(top_1000, doc_repres, 'lsi', param, 'lsi_ntopics='+str(param)+'.run')

creating file: run/lda_ntopics=10.run
creating LSM with 10 topics.
Retrieving completed.
creating file: run/lsi_ntopics=10.run
creating LSM with 10 topics.
Retrieving completed.
creating file: run/lda_ntopics=20.run
creating LSM with 20 topics.
Retrieving completed.
creating file: run/lsi_ntopics=20.run
creating LSM with 20 topics.
Retrieving completed.
creating file: run/lda_ntopics=50.run
creating LSM with 50 topics.
Retrieving completed.
creating file: run/lsi_ntopics=50.run
creating LSM with 50 topics.
Retrieving completed.
creating file: run/lda_ntopics=100.run
creating LSM with 100 topics.


/Users/davidstap/anaconda2/envs/ir1/lib/python3.5/site-packages/gensim/models/ldamodel.py:802: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Retrieving completed.
creating file: run/lsi_ntopics=100.run
creating LSM with 100 topics.
Retrieving completed.
creating file: run/lda_ntopics=200.run
creating LSM with 200 topics.
Retrieving completed.
creating file: run/lsi_ntopics=200.run
creating LSM with 200 topics.
Retrieving completed.


In [ ]:
trec_eval('ap_88_898_89/qrel_validation', 'run/lsi_top1000.run',mean=True)

### Task 3:  Word embeddings for ranking [20 points] (open-ended) ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).
   
This is an open-ended task. It is left up you to decide how you will combine word embeddings to derive query and document representations. Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt.tar.gz). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

def train_word2vec(corpus):
    '''
    This method uses gensim's implementation of word2vec to train it on
    a corpus constructed from a pyndri index. It saves and returns a 
    KeyedVectors object, which, when loaded appropriately, serves as a simple
    token to word embedding mapping.
    
    :param corpus: an iterable that returns a list of tokens for each document
    '''
    word2vec_model = Word2Vec(
        size=300,  # Embedding size
        window=5,  # One-sided window size
        sg=True,  # Skip-gram.
        min_count=5,  # Minimum word frequency.
        sample=1e-3,  # Sub-sample threshold.
        hs=False,  # Hierarchical softmax.
        negative=10,  # Number of negative examples.
        iter=5,  # Number of iterations.
        workers=8,  # Number of workers.
    )
    word2vec_model.build_vocab(corpus, trim_rule=None)
    word2vec_model.train(corpus, word2vec_model.corpus_count, epochs=word2vec_model.iter)
    word_vectors = word2vec.wv
    word_vectors.save('custom_word_vectors.txt')
    return word_vectors

corpus = MyCorpus(method = 'word2vec')
# word_vectors = train_word2vec(corpus)

# Load pre-trained word2vec embeddings from google
word_vectors = KeyedVectors.load_word2vec_format('reduced_vectors_google.txt', binary=False)

In [ ]:
def wordvecs_to_repr(word_vectors, pooling_method, doc_id = None, q_id = None):
    '''
    This method construct a fixed vector representation of a query
    or a document based on their individual word embeddings obtained
    by word2vec. There are two ways to get to those representations:
    the first is to simple average all word vectors in the query
    or document, the second is to take a weighted average, weighted
    by a word's TFIDF score.
    
    :param word_vectors: dict that maps token to its word2vec embedding
    :param pooling_method: tfidf or avr
    :param doc_id/q_id: either a query or doc id
    '''
    
    # Either use a query or doc_id
    if doc_id:
        unigram = docs_unigram[doc_id]
    elif q_id:
        unigram = collections.Counter(
                token_id for token_id in tokenized_queries[q_id]
                if token_id > 0)
    else:
        raise ValueError('Please specify either a doc id or a query id')
        
    # Start with a vector of zeros and add (weighted) word embeddings to it
    total_weight = 0
    weighted_bow = np.zeros(word_vectors['independent'].shape)
    for token_id, tf in unigram.items():
        try:
            word_vec = word_vectors[id2token[token_id]]
        except KeyError:
            continue
            
        if pooling_method == 'tfidf':
            idf = log(num_documents) - log(len(inverted_index[token_id]))
            tfidf = log(1+tf)*idf
            total_weight += tfidf
            weighted_bow += tfidf*word_vec
        elif pooling_method == 'avr':
            total_weight += 1
            weighted_bow += word_vec
            
    # Return a (weighted) average
    return weighted_bow/total_weight

pooling_method = 'tfidf'
wordvecs_to_repr(word_vectors, pooling_method, q_id='52')

### Task 4: Learning to rank (LTR) [15 points] (open-ended) ###

In this task you will get an introduction into learning to rank for information retrieval.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1, Task 2 and Task 3 as features. Think about other features you can use (e.g. query/document length). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

You are adviced to start some pointwise learning to rank algorithm e.g. logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set. More advanced learning to rank algorithms will be appreciated when grading.

### Task 4: Write a report [15 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.
